<a href="https://colab.research.google.com/github/kdemertzis/R2D2/blob/main/r2d2-final-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ipywidgets
!pip install scikit-fuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 KB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894086 sha256=8301ebe9363811b8737102c94252f218c94b96e260d072d79d74845cffb21cae
  Stored in directory: /root/.cache/pip/wheels/32/2c/a1/a90a7d7dd8448ec029f298a61f3490275e99b17aa348be675c
Successfully built scikit-fuzzy
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 889.2/889.2 KB 11.7 MB/s eta 0:00:00
  Preparing metadata

In [13]:
import requests
import json
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import skfuzzy as fuzz

def fetch_cvss_scores(cve_ids):
    base_url = "https://services.nvd.nist.gov/rest/json/cve/1.0/"
    cvss_scores = {}

    for cve_id in cve_ids:
        url = f"{base_url}{cve_id}"
        response = requests.get(url)

        if response.status_code == 200:
            data = json.loads(response.text)
            cvss_v3 = data["result"]["CVE_Items"][0]["impact"]["baseMetricV3"]["cvssV3"]["baseScore"]
            cvss_scores[cve_id] = cvss_v3
        else:
            print(f"Error fetching data for {cve_id}")

    return cvss_scores

def normalize_score(score, old_range, new_range):
    old_min, old_max = old_range
    new_min, new_max = new_range
    return (((score - old_min) * (new_max - new_min)) / (old_max - old_min)) + new_min

def calculate_threat_probability(cvss_scores, threat_frequencies):
    weighted_scores = {}
    for cve_id in cvss_scores:
        weighted_scores[cve_id] = normalize_score(cvss_scores[cve_id], (0, 10), (1, 10)) * threat_frequencies[cve_id]

    return weighted_scores

def fuzzy_categorize_threats(weighted_scores):
    x_score = np.arange(0, 101, 1)

    # Define fuzzy membership functions for Low, Medium, and High categories
    low = fuzz.trimf(x_score, [0, 0, 50])
    medium = fuzz.trimf(x_score, [25, 50, 75])
    high = fuzz.trimf(x_score, [50, 100, 100])

    categorized_threats = {}

    for cve_id, score in weighted_scores.items():
        low_category = fuzz.interp_membership(x_score, low, score)
        medium_category = fuzz.interp_membership(x_score, medium, score)
        high_category = fuzz.interp_membership(x_score, high, score)

        max_category = max(low_category, medium_category, high_category)

        if max_category == low_category:
            categorized_threats[cve_id] = ("Low", score)
        elif max_category == medium_category:
            categorized_threats[cve_id] = ("Medium", score)
        else:
            categorized_threats[cve_id] = ("High", score)

    return categorized_threats


def fuzzy_risk(risk_score):
    x_risk = np.arange(0, 101, 1)

    # Define fuzzy membership functions for Low, Medium, and High risk categories
    low_risk = fuzz.trimf(x_risk, [0, 0, 50])
    medium_risk = fuzz.trimf(x_risk, [25, 50, 75])
    high_risk = fuzz.trimf(x_risk, [50, 100, 100])

    low_risk_category = fuzz.interp_membership(x_risk, low_risk, risk_score)
    medium_risk_category = fuzz.interp_membership(x_risk, medium_risk, risk_score)
    high_risk_category = fuzz.interp_membership(x_risk, high_risk, risk_score)

    max_category = max(low_risk_category, medium_risk_category, high_risk_category)

    if max_category == low_risk_category:
        return "Low"
    elif max_category == medium_risk_category:
        return "Medium"
    else:
        return "High"


def on_calculate_button_clicked(b):
    cve_ids = cve_input.value.split(",")
    threat_frequencies = {cve_id.strip(): freq_slider.value for cve_id in cve_ids}
    impact_scores = {cve_id.strip(): normalize_score(float(impact), (0, 10), (1, 10)) for cve_id, impact in zip(cve_ids, map(float, impact_input.value.split(",")))}

    cvss_scores = fetch_cvss_scores(cve_ids)
    weighted_scores = calculate_threat_probability(cvss_scores, threat_frequencies)
    categorized_threats = fuzzy_categorize_threats(weighted_scores)

    print("Categorized Threat Probabilities:")
    for cve_id, (category, score) in categorized_threats.items():
        print(f"{cve_id}: {category} ({score:.2f})")

    risks = {}
    for cve_id in cve_ids:
        impact = impact_scores[cve_id.strip()]
        likelihood = weighted_scores[cve_id.strip()]
        criticality = normalize_score(cvss_scores[cve_id.strip()], (0, 10), (1, 10))
        risk = normalize_score(impact * likelihood * criticality, (1, 1000), (1, 10))
        risks[cve_id.strip()] = risk

    print("Risks:")
    for cve_id, risk in risks.items():
        linguistic_risk = fuzzy_risk(risk)
        print(f"{cve_id}: {linguistic_risk} ({risk:.2f})")


# Create input widgets
cve_input = widgets.Text(
    placeholder="Enter CVE IDs separated by commas",
    description="CVE IDs:",
    layout=widgets.Layout(width="70%")
)

impact_slider = widgets.IntSlider(
    value=5,
    min=1,
    max=10,
    step=1,
    description="Asset Impact:",
    continuous_update=False
)

freq_slider = widgets.IntSlider(
    value=5,
    min=1,
    max=10,
    step=1,
    description="Frequency:",
    continuous_update=False
)

calculate_button = widgets.Button(
    description="Calculate",
    button_style="info",
    tooltip="Click to calculate threat probabilities and risks"
)

# Display widgets
display(cve_input)
display(impact_slider)
display(freq_slider)
display(calculate_button)

# Assign button click event
calculate_button.on_click(on_calculate_button_clicked)

Text(value='', description='CVE IDs:', layout=Layout(width='70%'), placeholder='Enter CVE IDs separated by com…

IntSlider(value=5, continuous_update=False, description='Asset Impact:', max=10, min=1)

IntSlider(value=5, continuous_update=False, description='Frequency:', max=10, min=1)

Button(button_style='info', description='Calculate', style=ButtonStyle(), tooltip='Click to calculate threat p…

Categorized Threat Probabilities:
CVE-2022-33936: High (88.38)
Risks:
CVE-2022-33936: Medium (43.99)
Categorized Threat Probabilities:
CVE-2022-33936: High (88.38)
Risks:
CVE-2022-33936: Medium (43.99)
Categorized Threat Probabilities:
CVE-2022-33936: High (98.20)
Risks:
CVE-2022-33936: Medium (48.77)
Categorized Threat Probabilities:
CVE-2022-33936: Low (29.46)
Risks:
CVE-2022-33936: Low (15.33)
Categorized Threat Probabilities:
CVE-2022-33936: Low (29.46)
Risks:
CVE-2022-33936: Low (15.33)
Categorized Threat Probabilities:
CVE-2022-33936: Low (29.46)
Risks:
CVE-2022-33936: Low (15.33)
Categorized Threat Probabilities:
CVE-2022-33936: Medium (49.10)
Risks:
CVE-2022-33936: Low (24.88)
Categorized Threat Probabilities:
CVE-2022-33936: High (98.20)
Risks:
CVE-2022-33936: Medium (48.77)
Categorized Threat Probabilities:
CVE-2022-33936: High (98.20)
Risks:
CVE-2022-33936: Medium (48.77)
